En este notebook se realiza efectivamente el entrenamiento. 

Se unifican las clases 0-4. Se crean 2 pipelines, eligiendo finalmente el que utiliza un standar scaler para las variables numéricas. Una vez transformada la data, se hace un oversampling con SMOTE. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..") 

In [63]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, make_union
from utils.transformers import FillNaDict, DFFeatureUnion, ColumnSelector, DFStandardScaler, DummyTransformer, DFKBinsOrdinalDiscretizer, DFOrdinalEncoder
import numpy as np
from sklearn.model_selection import GridSearchCV
import joblib
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

## Config

In [5]:
seed = 1111

In [6]:
data_path = "data/"
input_path = data_path + "input/"
models_path = data_path + "models/"
columns_path = data_path + "columns/"
pipelines_path = data_path + "pipelines/"

## Load data


In [7]:
df = pd.read_csv(input_path + 'train.csv', sep='\t', encoding='utf-8', )  
df.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,is_official_store,catalog_product_id,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,address.state_name,shipping.free_shipping,shipping.mode,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name
0,12.0,70.17,MLA901604420,MLA,Hacha - Bendy And The Ink Machine - Prop,1800.00,ARS,50,25,buy_it_now,gold_special,new,True,MLA3422,False,NaN,NaN,132586939,False,False,694.0,32.0,historic,0.00,0.97,0.03,662.0,silver,0.0000,0.0000,60 days,291.0,0.0000,Buenos Aires,False,me2,xd_drop_off,False,NaN,MLA1132,Juegos y Juguetes,NaN,Muñecos y Figuras de Acción
1,12.0,70.17,MLA785203596,MLA,Microscopio Lupa Celular Tricomas 60x Led Gabb...,1199.00,ARS,1,500,buy_it_now,gold_special,new,True,MLA32230,False,NaN,NaN,401908084,False,False,25441.0,943.0,historic,0.01,0.99,0.00,24498.0,platinum,0.0020,0.0000,60 days,3267.0,0.0000,Buenos Aires,False,me2,cross_docking,False,NaN,MLA1039,Cámaras y Accesorios,NaN,Microscopios
2,12.0,70.17,MLA703379424,MLA,Chapa De Madera De Guatambú - Lamina Para Ench...,420.00,ARS,1,250,buy_it_now,gold_special,new,True,MLA1383,False,NaN,NaN,50806723,False,False,298.0,13.0,historic,0.00,1.00,0.00,285.0,silver,0.0000,0.0000,60 days,72.0,0.0000,Buenos Aires,False,me2,xd_drop_off,False,green,MLA1367,Antigüedades y Colecciones,NaN,Otros
3,12.0,70.17,MLA647580156,MLA,Bomba Electrica De Combustible Nafta O Gasoil ...,2426.00,ARS,500,250,buy_it_now,gold_special,new,True,MLA417251,False,NaN,NaN,119023482,False,False,7841.0,589.0,historic,0.00,0.99,0.01,7252.0,platinum,0.0134,0.0085,60 days,1466.0,0.0032,Buenos Aires,False,me2,cross_docking,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Bombas de Combustible
4,12.0,70.17,MLA762063286,MLA,Juego X 4 Tazas Vw Gol / Gol Trend Rodado 14 P...,2439.99,ARS,1,500,buy_it_now,gold_special,new,True,MLA9250,False,NaN,NaN,173563910,False,False,66601.0,2775.0,historic,0.01,0.98,0.01,63826.0,platinum,0.0074,0.0158,60 days,11171.0,0.0022,Buenos Aires,False,me2,fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Tazas


In [9]:
df.shape

(23180, 43)

## Target

In [16]:
df.sold_quantity.value_counts().sort_index()

0        2325
1         212
2         187
3         173
4         161
5        1740
25       1379
50       2105
100      1599
150      1291
200      1086
250      3541
500      6859
5000      514
50000       8
Name: sold_quantity, dtype: int64

## Manipulación clases

<p style="color:red; font-weight: bold">Vamos a intentar probar unificar las categorias iniciales, considerando este tipo de decisiones, en caso de quen hagan que el modelo de mejor, deben consultarse siempre con el negocio </p>

In [17]:
df["sold_quantity"] = [x if x >= 5 else 1 for x in df["sold_quantity"]]

In [18]:
df.sold_quantity.value_counts().sort_index()

1        3058
5        1740
25       1379
50       2105
100      1599
150      1291
200      1086
250      3541
500      6859
5000      514
50000       8
Name: sold_quantity, dtype: int64

# y

In [19]:
y = df.sold_quantity

In [20]:
df = df.drop(['sold_quantity'], axis= 1)

## SMOTE strategy

In [21]:
dic = pd.Series(y).value_counts().sort_index().to_dict()
dic

{1: 3058,
 5: 1740,
 25: 1379,
 50: 2105,
 100: 1599,
 150: 1291,
 200: 1086,
 250: 3541,
 500: 6859,
 5000: 514,
 50000: 8}

In [22]:
threshold = 3000
prop = 1.5

In [23]:
smote_sampling_strategy = {key: int(value*prop) if value < threshold else value for key, value in dic.items()}
smote_sampling_strategy

{1: 3058,
 5: 2610,
 25: 2068,
 50: 3157,
 100: 2398,
 150: 1936,
 200: 1629,
 250: 3541,
 500: 6859,
 5000: 771,
 50000: 12}

## IDs  

In [24]:
id_cols = joblib.load(columns_path + 'id_cols.joblib')
id_cols

['id',
 'site_id',
 'title',
 'category_id',
 'seller.id',
 'catalog_product_id',
 'root_category_id',
 'root_category_name',
 'category_name']

In [25]:
ids_df = df[id_cols].copy()
X = df.drop(id_cols, axis= 1) 

## Columns

In [26]:
X.dtypes

installments.quantity                                          float64
installments.rate                                              float64
price                                                          float64
currency_id                                                     object
available_quantity                                               int64
buying_mode                                                     object
listing_type_id                                                 object
condition                                                       object
accepts_mercadopago                                               bool
is_official_store                                                 bool
catalog_listing                                                 object
seller.car_dealer                                                 bool
seller.real_estate_agency                                         bool
seller.seller_reputation.transactions.total                    float64
seller

In [27]:
cols_preproc = X.columns
cols_preproc

Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_reputation.transactions.total',
       'seller.seller_reputation.transactions.canceled',
       'seller.seller_reputation.transactions.period',
       'seller.seller_reputation.transactions.ratings.negative',
       'seller.seller_reputation.transactions.ratings.positive',
       'seller.seller_reputation.transactions.ratings.neutral',
       'seller.seller_reputation.transactions.completed',
       'seller.seller_reputation.power_seller_status',
       'seller.seller_reputation.metrics.claims.rate',
       'seller.seller_reputation.metrics.delayed_handling_time.rate',
       'seller.seller_reputation.metrics.sales.period',
       'seller.seller_reputation.metrics.sales.completed',
       

In [28]:
categorical_ord_enc = [
    'available_quantity',
] + joblib.load(columns_path + 'bool_cols.joblib')
categorical_ord_enc

['available_quantity',
 'accepts_mercadopago',
 'is_official_store',
 'catalog_listing',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'shipping.free_shipping',
 'shipping.store_pick_up']

In [29]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).drop(categorical_ord_enc, axis = 1, errors='ignore').columns.to_list()
numeric_features

['installments.quantity',
 'installments.rate',
 'price',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.sales.completed',
 'seller.seller_reputation.metrics.cancellations.rate',
 'discount']

In [30]:
categorical_features = X.select_dtypes(include=['object']).drop(categorical_ord_enc, axis = 1, errors='ignore').columns.to_list()
categorical_features

['currency_id',
 'buying_mode',
 'listing_type_id',
 'condition',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.sales.period',
 'address.state_name',
 'shipping.mode',
 'shipping.logistic_type',
 'seller.seller_reputation.real_level']

## NA

Nota: habría que consultar con expertos del dominio en cómo completar; en este caso, segun el tipo, van con un valor fijo

In [31]:
fillNA = {
    'cat': {},
    'num': {},
    'ord': {}
}
for x in categorical_ord_enc:
    fillNA['ord'][x] = 0 

for x in numeric_features:
    fillNA['num'][x] = 0 
    
for x in categorical_features:
    fillNA['cat'][x] = '__NA__'

In [32]:
fillNA

{'cat': {'currency_id': '__NA__',
  'buying_mode': '__NA__',
  'listing_type_id': '__NA__',
  'condition': '__NA__',
  'seller.seller_reputation.transactions.period': '__NA__',
  'seller.seller_reputation.power_seller_status': '__NA__',
  'seller.seller_reputation.metrics.sales.period': '__NA__',
  'address.state_name': '__NA__',
  'shipping.mode': '__NA__',
  'shipping.logistic_type': '__NA__',
  'seller.seller_reputation.real_level': '__NA__'},
 'num': {'installments.quantity': 0,
  'installments.rate': 0,
  'price': 0,
  'seller.seller_reputation.transactions.total': 0,
  'seller.seller_reputation.transactions.canceled': 0,
  'seller.seller_reputation.transactions.ratings.negative': 0,
  'seller.seller_reputation.transactions.ratings.positive': 0,
  'seller.seller_reputation.transactions.ratings.neutral': 0,
  'seller.seller_reputation.transactions.completed': 0,
  'seller.seller_reputation.metrics.claims.rate': 0,
  'seller.seller_reputation.metrics.delayed_handling_time.rate': 0,


## Pipelines
Notas: 
- podría haber elegido diferente preprocesamiento para variables de un mismo tipo
- los transformers desarrollados son "wrappers" de aquellos provistos por sklearn, para trabajar con pandas data frames en vez de np arrays

In [33]:
categorical_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(categorical_features)),
    ('imputer', FillNaDict(col_fill_dict = fillNA['cat'])),
    ('dummies', DummyTransformer(categorical_features))
    ])


cat_label_encod = Pipeline(steps=[
    ('extractor',ColumnSelector(categorical_ord_enc)),
    ('imputer', FillNaDict(col_fill_dict = fillNA['ord'])),
    ('ordinalEncoder',DFOrdinalEncoder(categorical_ord_enc))
])

sel_ini_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(cols_preproc))
])


**Pipeline con binarizer para variables numericas**

In [34]:
numeric_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict = fillNA['num'])),
    ('binarizer',DFKBinsOrdinalDiscretizer(n_bins=5,cols=numeric_features))
    ])


# DFFeatureUnion
feat_un = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('cat_lab_enc',cat_label_encod),
            ('numeric',numeric_transformer),
            ])

# pipeline_base
pipeline_base = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un)])
pipeline_base

Pipeline(steps=[('seleccionFeatsIni',
                 Pipeline(steps=[('extractor',
                                  ColumnSelector(columns=Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_...
       'seller.seller_reputation.metrics.sales.completed',
       'seller.seller_reputation.metrics.cancellations.rate',
       'address.state_name', 'shipping.free_shipping', 'shipping.mode',
       'shipping.logistic_type', 'shipping.store_pick_up',
       'seller.seller_reputation.real_level', 'discount'],
      dtype='object')))])),
                ('final_features',
                 <utils.transformers.DFFeatureUnion object at 0x000001EB86F450A0>)])

**Pipeline con scaler para vars numéricas**

In [35]:
numeric_transformer_w_scaler = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict= fillNA['num'])),
    ('scaler',DFStandardScaler())
    ])

# DFFeatureUnion
feat_un2 = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('cat_lab_enc',cat_label_encod),
            ('numeric',numeric_transformer_w_scaler),
            ])

# pipeline_w_num_scaler
pipeline_w_num_scaler = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un2)])

In [36]:
pipeline_w_num_scaler

Pipeline(steps=[('seleccionFeatsIni',
                 Pipeline(steps=[('extractor',
                                  ColumnSelector(columns=Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_...
       'seller.seller_reputation.metrics.sales.completed',
       'seller.seller_reputation.metrics.cancellations.rate',
       'address.state_name', 'shipping.free_shipping', 'shipping.mode',
       'shipping.logistic_type', 'shipping.store_pick_up',
       'seller.seller_reputation.real_level', 'discount'],
      dtype='object')))])),
                ('final_features',
                 <utils.transformers.DFFeatureUnion object at 0x000001EBA9650670>)])

## Transformación con pipeline elegido

In [37]:
X = X.reset_index(drop=True)

In [38]:
pipeline = pipeline_w_num_scaler # pipeline_base

In [39]:
y_copy = y.copy()
X_copy = X.copy()

In [40]:
X_copy = pipeline.fit_transform(X_copy) 
X_copy.reset_index(drop=True, inplace= True)


In [41]:
path_pipeline = pipelines_path + 'pipeline.joblib'
joblib.dump(pipeline, path_pipeline)


['data/pipelines/pipeline.joblib']

**Final cols**

In [42]:
columns_names = X_copy.columns
columns_names.tolist()

['listing_type_id_bronze',
 'listing_type_id_gold',
 'listing_type_id_gold_premium',
 'listing_type_id_gold_pro',
 'listing_type_id_gold_special',
 'listing_type_id_silver',
 'seller.seller_reputation.power_seller_status___NA__',
 'seller.seller_reputation.power_seller_status_gold',
 'seller.seller_reputation.power_seller_status_platinum',
 'seller.seller_reputation.power_seller_status_silver',
 'currency_id_ARS',
 'currency_id_USD',
 'currency_id___NA__',
 'shipping.mode_custom',
 'shipping.mode_me1',
 'shipping.mode_me2',
 'shipping.mode_not_specified',
 'condition___NA__',
 'condition_new',
 'condition_not_specified',
 'condition_used',
 'buying_mode_buy_it_now',
 'buying_mode_classified',
 'address.state_name_Bs.As. Costa Atlántica',
 'address.state_name_Bs.As. G.B.A. Norte',
 'address.state_name_Bs.As. G.B.A. Oeste',
 'address.state_name_Bs.As. G.B.A. Sur',
 'address.state_name_Buenos Aires',
 'address.state_name_Buenos Aires Interior',
 'address.state_name_Capital Federal',
 'add

In [43]:
path_columns = columns_path + 'final_cols.joblib'
joblib.dump(columns_names, path_columns)

['data/columns/final_cols.joblib']

**Inspect df**

In [44]:
X_copy.head()

,listing_type_id_bronze,listing_type_id_gold,listing_type_id_gold_premium,listing_type_id_gold_pro,listing_type_id_gold_special,listing_type_id_silver,seller.seller_reputation.power_seller_status___NA__,seller.seller_reputation.power_seller_status_gold,seller.seller_reputation.power_seller_status_platinum,seller.seller_reputation.power_seller_status_silver,currency_id_ARS,currency_id_USD,currency_id___NA__,shipping.mode_custom,shipping.mode_me1,shipping.mode_me2,shipping.mode_not_specified,condition___NA__,condition_new,condition_not_specified,condition_used,buying_mode_buy_it_now,buying_mode_classified,address.state_name_Bs.As. Costa Atlántica,address.state_name_Bs.As. G.B.A. Norte,address.state_name_Bs.As. G.B.A. Oeste,address.state_name_Bs.As. G.B.A. Sur,address.state_name_Buenos Aires,address.state_name_Buenos Aires Interior,address.state_name_Capital Federal,address.state_name_Catamarca,address.state_name_Chaco,address.state_name_Chubut,address.state_name_Corrientes,address.state_name_Córdoba,address.state_name_Entre Ríos,address.state_name_Formosa,address.state_name_Jujuy,address.state_name_La Pampa,address.state_name_La Rioja,address.state_name_Mendoza,address.state_name_Misiones,address.state_name_Neuquén,address.state_name_Río Negro,address.state_name_Salta,address.state_name_San Juan,address.state_name_San Luis,address.state_name_Santa Cruz,address.state_name_Santa Fe,address.state_name_Santiago del Estero,address.state_name_Tierra del Fuego,address.state_name_Tucumán,seller.seller_reputation.metrics.sales.period_365 days,seller.seller_reputation.metrics.sales.period_60 days,seller.seller_reputation.metrics.sales.period_60 months,seller.seller_reputation.metrics.sales.period___NA__,seller.seller_reputation.real_level___NA__,seller.seller_reputation.real_level_green,seller.seller_reputation.real_level_green_gold,seller.seller_reputation.real_level_green_platinum,seller.seller_reputation.real_level_green_silver,seller.seller_reputation.real_level_orange,seller.seller_reputation.real_level_red,seller.seller_reputation.real_level_yellow,seller.seller_reputation.transactions.period___NA__,seller.seller_reputation.transactions.period_historic,shipping.logistic_type___NA__,shipping.logistic_type_cross_docking,shipping.logistic_type_custom,shipping.logistic_type_default,shipping.logistic_type_drop_off,shipping.logistic_type_fulfillment,shipping.logistic_type_not_specified,shipping.logistic_type_xd_drop_off,available_quantity,accepts_mercadopago,is_official_store,catalog_listing,seller.car_dealer,seller.real_estate_agency,shipping.free_shipping,shipping.store_pick_up,installments.quantity,installments.rate,price,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,discount
0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0.292067,0.432591,-0.110504,-0.556285,-0.479515,-0.749099,0.165588,0.348945,-0.558724,-0.784924,-0.594933,-0.455433,-0.651310,-0.301447
1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.292067,0.432591,-0.111038,-0.202483,-0.262958,-0.189255,0.278190,-0.313309,-0.197738,-0.517923,-0.594933,-0.267342,-0.651310,-0.301447
2,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0.292067,0.432591,-0.111729,-0.561947,-0.484032,-0

In [46]:
[x for x in X_copy.columns if "off" in x]

['shipping.logistic_type_drop_off',
 'shipping.logistic_type_xd_drop_off',
 'is_official_store']

In [47]:
X_copy.discount.value_counts()

-0.301447    20364
 0.411597      296
 1.124640      131
 0.411597       94
 0.554205       72
             ...  
 0.411779        1
 0.577973        1
 3.905405        1
 4.689968        1
 3.976815        1
Name: discount, Length: 1394, dtype: int64

In [48]:
X_copy['installments.quantity'].value_counts()

 0.292067    19764
-3.372904     1673
-0.624176     1326
 2.124553      376
-2.456662       41
Name: installments.quantity, dtype: int64

In [49]:
X_copy['price'].value_counts()

-0.112102    183
-0.111215    140
-0.108995    138
-0.110328    131
-0.111347    123
            ... 
-0.111395      1
-0.110619      1
-0.111816      1
-0.111782      1
-0.105765      1
Name: price, Length: 7571, dtype: int64

In [50]:
X.available_quantity.value_counts()

1        13344
50        2653
500       2466
250       1508
100       1083
150        758
5000       658
200        458
50000      252
Name: available_quantity, dtype: int64

In [51]:
X_copy.available_quantity.value_counts()

0    13344
1     2653
6     2466
5     1508
2     1083
3      758
7      658
4      458
8      252
Name: available_quantity, dtype: int64

In [52]:
X.accepts_mercadopago.value_counts()

True     22457
False      723
Name: accepts_mercadopago, dtype: int64

In [53]:
X_copy.accepts_mercadopago.value_counts()

1    22457
0      723
Name: accepts_mercadopago, dtype: int64

## Oversampling con smote

In [54]:
oversample = SMOTE(n_jobs=7,
                  sampling_strategy= smote_sampling_strategy)
X_copy, y_copy = oversample.fit_resample(X_copy, y_copy)


Instancias por clase post resampling:

In [55]:
y_copy.value_counts().sort_index()

1        3058
5        2610
25       2068
50       3157
100      2398
150      1936
200      1629
250      3541
500      6859
5000      771
50000      12
Name: sold_quantity, dtype: int64

## Models
Nota: podría haber probado con más modelos (KNN, Random Forest, Decision Tree, ... )

In [56]:
models = {         
          'lr': LogisticRegression(),
          'xgb':XGBClassifier(),
          'mlp':MLPClassifier(),
}

In [57]:
hparams = {'xgb':{'max_depth':[5,7,10], 'gamma': [1,5,10,20], 'n_jobs':[7], 'learning_rate': [0.05, 0.1], 'n_estimators':[100, 200, 500],
                 'colsample_bytree': [0.2, 0.5], 'seed': [seed]},
     
          'lr': {'multi_class': ['ovr'], 'class_weight':['balanced'], 'C': [0.01, 0.1, 1, 10,], 'solver': ['saga',], 'random_state':[seed]},
    
          
          'mlp':{'solver':["adam", 'sgd'], 'hidden_layer_sizes' : [(64, 64, 64), (64, 64, 64, 64), (64, 64, 64, 64, 64)], 'activation' : ['relu'], 'learning_rate_init' : [0.001, 0.01, 0.05], 'alpha' : [0.0001, 0.05, 0.001],
                 'max_iter':[1800], 'batch_size':[64, 256], 'learning_rate': ['constant','adaptive'], 'random_state':[seed]},         
     
}

In [58]:
diccionario_entrenamiento = {
                            'xgb': {},
                            'lr': {},
                            'mlp': {},
}
                                                         
for valor in diccionario_entrenamiento.keys():
    diccionario_entrenamiento[valor]['model'] = models[valor]
    diccionario_entrenamiento[valor]['params'] = hparams[valor]


## Entrenamiento
Nota: or defecto, el Grid Search reentrena con todo el dataset, usando los mejores hyper parámetros encontrados (refit = True)

In [59]:
cv_n = 3 

In [60]:
models_path_template = models_path + '{}.joblib'

In [61]:
basic_opt_params = {'scoring': 'f1_macro', 'cv': cv_n, 'n_jobs': 7}

In [62]:
for valor in tqdm(diccionario_entrenamiento.keys()):
    model = diccionario_entrenamiento[valor]['model']
    params = diccionario_entrenamiento[valor]['params']
    
    opt = GridSearchCV(estimator=model, param_grid=params, )
    opt.set_params(**basic_opt_params)
    fit = opt.fit(X_copy[columns_names], y_copy)
    
    # persisto el best estimator para cada uno
    model_name = models_path_template.format(valor)
    if valor == 'xgb':
        fit.best_estimator_.save_model(model_name)
    else:
        joblib.dump(fit.best_estimator_, model_name)

  0%|          | 0/3 [00:00<?, ?it/s]

c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:48:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users